# AMOS 2024 - BARPA - Australian annual mean temperature

In this notebook we explore the annual mean warming of the dynamically downscaled BARPA-R simulations. The goal is to load the historical and ssp370 scenario for one downscaled CMIP6 model and plot the annual mean temperature over time.

In [ ]:
import os
import nci_ipynb
os.chdir(nci_ipynb.dir())
print(os.getcwd())

First we load the required python modules and start a dask client to speed up our computation.

In [ ]:
import tempfile
import dask.distributed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys, os

from esmloader import EsmCat

In [ ]:
# Specify the which data collection you want to load, i.e. BARRA2 or BARPA.
barpa=EsmCat("BARPA")

In [ ]:
client = dask.distributed.Client(local_directory=tempfile.mkdtemp())
client

Now that we have a client (click on "Launch dashboard in JupyterLab" to see the dask dashboard) running lets open the historical and ssp370 files for ACCESS-CM2 downscaled simulation. For this we use the provided load function function with parallel set to true. 

You can see the progress of opening the files in the dask dashboard.

***
## Introduction

In [ ]:
ds_tas_hist = barpa.load_barpa_data(
                'BARPA-R', 
                'ACCESS-CM2', 
                'historical', 
                'mon', 
                'tas'
)
ds_tas_ssp = barpa.load_barpa_data(
                'BARPA-R', 
                'ACCESS-CM2', 
                'ssp370', 
                'mon', 
                'tas'
)

We can also inspect the dataset we have just opened. If you click on "Attributes" you can get much more information about the content of the files you have just opened, such as information about the driving model, the native resolution of BARPA-R, etc.

In [ ]:
ds_tas_hist

Instead of two individual datasets for the historical and future scenario it is much easier to work with one combined dataset. For this we use xarray's concat function to concatenate the two datasets along the time dimension.

In [ ]:
ds_tas = xr.concat(
    [ds_tas_hist, ds_tas_ssp],
    dim="time"
)

Also we are only interested in the near-surface temperature (tas) of the dataset. Therefore we extract "tas" from the dataset and turn it into a dataarray. The advantage of this is that datasets cannot be plotted directly but dataarrays can.

In [ ]:
da_tas = ds_tas["tas"]

Next we perform some calculations. To plot the annual mean temperature over time we need to calculate the area mean of the domain as well as the yearly mean and then pass this to the plot routine.

In [ ]:
da_tas_mn = da_tas.mean(["lat", "lon"])

In [ ]:
da_tas_mn = da_tas_mn.resample({"time":"1Y"}).mean()

In [ ]:
da_tas_mn.plot()

***
## Exercise 1 (optional)
BARPA-R has downscaled 7 CMIP6 models, which are listed below. In the introduction we only plotted the annual mean of one model, ACCESS-CM2. Extending on the above code, can you plot the annual and area mean for all 7 models over time?

Hint: A loop over the model_list below and modifying the data loader will help you to load all the monthly files you need.

In [ ]:
model_list = [
    "ACCESS-CM2",
    "ACCESS-ESM1-5",
    "CESM2",
    "CMCC-ESM2",
    "EC-Earth3",
    "MPI-ESM1-2-HR",
    "NorESM2-MM",
]

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
dict_da_tas_model = {}
for model in model_list:
    # Load historical data
    ds_tas_hist = barpa.load_barpa_data(
                'BARPA-R', 
                model, 
                'historical', 
                'mon', 
                'tas'
    )
    # Load ssp data
    ds_tas_ssp = barpa.load_barpa_data(
                'BARPA-R', 
                model, 
                'ssp370', 
                'mon', 
                'tas'
    )
    
    dict_da_tas_model[model] = xr.concat(
        [ds_tas_hist, ds_tas_ssp],
        dim="time"
    )["tas"]

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
dict_da_tas_model_mn = {}
for model in model_list:
    dict_da_tas_model_mn[model] = dict_da_tas_model[model].mean(["lat", "lon"]).resample({"time":"1Y"}).mean().compute()

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
for model in model_list:
    plt.plot(dict_da_tas_model_mn[model]["time"].dt.year, dict_da_tas_model_mn[model], label=model)
    plt.legend()
    plt.title("Annual mean temperature")

***
## Exercise 2 (optional)
Now we extend our analysis by looking at an additional variable, precipitation.
Can you plot the annual maximum hourly precipitation rate over time? 

Hint: The variable to use for this is under the frequency directory "day" (instead of "mon") and is called "prhmax".

Hint 2: Opening and performing computations on daily files will take longer and more notebook resources might be required; you are opening 34GB of data.

Hint 3: Add chunks={"time":512, "lat":None, "lon":None} as argument to open_mfdataset. This will help load your data faster. It will still take several minutes to process the data for all 7 models.

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
dict_da_prhmax_model = {}
for model in model_list:
    # Load historical data
    ds_prhmax_hist = barpa.load_barpa_data(
                'BARPA-R', 
                model, 
                'historical', 
                'day', 
                'prhmax',
                chunks={"time":512, "lat":None, "lon":None}
    )
    # Load ssp data
    ds_prhmax_ssp = barpa.load_barpa_data(
                'BARPA-R', 
                model, 
                'ssp370', 
                'day', 
                'prhmax',
                chunks={"time":512, "lat":None, "lon":None}
    )
    
    dict_da_prhmax_model[model] = xr.concat(
        [ds_prhmax_hist, ds_prhmax_ssp],
        dim="time"
    )["prhmax"]

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
dict_da_prhmax_model_mn = {}
for model in model_list:
    dict_da_prhmax_model_mn[model] = dict_da_prhmax_model[model].max(["lat", "lon"]).resample({"time":"1Y"}).max().compute()

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
for model in model_list:
    plt.plot(dict_da_prhmax_model_mn[model]["time"].dt.year, dict_da_prhmax_model_mn[model], label=model)
    plt.legend()
    plt.title("Annual maximum hourly precipitation rate")

***
# Exercise 3 (optional)
In this exercise we are plotting the Ed Hawkins climate stripes (#ShowYourStripes) for the ensemble mean of all BARPA-R downscaled CMIP6 models. 

First you will need to calculate the annual mean, which you have done before, and the ensemble mean of all models.

Next you need to plot this in the style of the climate stripes. Have a look [here](https://showyourstripes.info) what they look like.

Hint: To calculate the ensemble mean it can be helpful to concatenate the dictionary of individual models into a single xarray dataset. To do this you can use xr.concat. The xarray documentation is also helpful. Have a look [here](https://docs.xarray.dev/en/stable/generated/xarray.concat.html). 

Hint 2: Because some BARPA models use different calendars it might be useful to overwrite the time coordinate with the year integer. For this you can use the assign_coords function.

Hint 3: You can use plt.bar to get very close to what the climate stripes look like.

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
dict_da_tas_model = {}
for model in model_list:
    # Load historical data
    ds_tas_hist = barpa.load_barpa_data(
                'BARPA-R', 
                model, 
                'historical', 
                'mon', 
                'tas'
    )
    # Load ssp data
    ds_tas_ssp = barpa.load_barpa_data(
                'BARPA-R', 
                model, 
                'ssp370', 
                'mon', 
                'tas'
    )
    
    dict_da_tas_model[model] = xr.concat(
        [ds_tas_hist, ds_tas_ssp],
        dim="time"
    )["tas"]

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
dict_da_tas_model_mn = {}
for model in model_list:
    dict_da_tas_model_mn[model] = dict_da_tas_model[model].mean(["lat", "lon"]).resample({"time":"1Y"}).mean().compute()

In [ ]:
for model in model_list:
    dict_da_tas_model_mn[model] = dict_da_tas_model_mn[model].assign_coords(
        time=dict_da_tas_model_mn[model].time.dt.year
    )

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
da_tas_model = xr.concat(
    [dict_da_tas_model_mn[model] for model in model_list],
    pd.Index(model_list, name="model")
)
da_tas_ensmean = da_tas_model.mean("model")

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
def climate_stripes(da, cmap="RdBu_r"):
    fig = plt.figure(figsize=plt.figaspect(0.5))
    cmap = plt.cm.get_cmap(cmap)
    norm = plt.Normalize(da.min(), da.max())
    colors = cmap(norm(da))
    plt.bar(da["time"], 1, width=0.95, color=colors)
    plt.axis('off')

In [ ]:
# !!! REMOVE FOR EXERCISE !!!
climate_stripes(da_tas_ensmean)

In [ ]:
client.close()